# Introduction

This notebook contains some examples of TimeAtlas usages.

## Installation

Installing the basic package (without heavy dependancies)

```pip install timeatlas```

for additional dependandies use:

 ```pip install timeatlas[torch]``` including [pytorch](https://pytorch.org/)

 ```pip install timeatlas[prophet]``` including [facebook prophet](https://facebook.github.io/prophet/)

  ```pip install timeatlas[all]``` including both


## Work with TimeSeries

The two essential components are a Pandas DataFrame and a DatetimeIndex

In [49]:
from timeatlas import TimeSeries
from pandas import DataFrame, DatetimeIndex

index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04','2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08','2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12'])
my_series = DataFrame([0.4, 1.0, 0.7, 0.6, 0.4, 1.0, 0.7, 0.6, 0.4, 1.0, 0.7, 0.6], index=index)

ts = TimeSeries(my_series)

In [50]:
ts

            values
2019-01-01     0.4
2019-01-02     1.0
2019-01-03     0.7
2019-01-04     0.6
2019-01-05     0.4
2019-01-06     1.0
2019-01-07     0.7
2019-01-08     0.6
2019-01-09     0.4
2019-01-10     1.0
2019-01-11     0.7
2019-01-12     0.6

You can check its main characteristics with `TimeSeries.describe()`

In [51]:
ts.describe()

count    12.000000
mean      0.675000
std       0.226134
min       0.400000
25%       0.550000
50%       0.650000
75%       0.775000
max       1.000000
Name: values, dtype: float64

And visualize it with the `TimeSeries.plot()` function

In [52]:
ts.plot()

## Add Metadata to a TimeSeries

In [53]:
from timeatlas import Metadata, types

# You can use some typed metadata object
my_unit = types.Unit("power", "W", "float")
my_sensor = types.Sensor(2902, "HB/floor2/22-23C/Prises_Power_Tot")
my_coords = types.Coords(46.796611, 7.147563)

# Or use Python dictionaries
my_location = {
    "building" : "Blue Factory",
    "floor" : "12",
    "room"  : "22C"
}

my_dict = {
    "unit": my_unit,
    "sensor": my_sensor,
    "location": my_location,
    "coordinates": my_coords
}

# Create the Metadata object
my_meta = Metadata(my_dict)

In [54]:
my_meta

{'unit': <timeatlas.types.unit.Unit at 0x7fba64e20c10>,
 'sensor': Sensor ID: 2902; Name: HB/floor2/22-23C/Prises_Power_Tot,
 'location': {'building': 'Blue Factory', 'floor': '12', 'room': '22C'},
 'coordinates': 46.796611°N, 7.147563°E}

## Define a TimeSeries with Metadata

In [55]:
ts_meta = TimeSeries(my_series, my_meta)

In [56]:
ts_meta.plot()

Create TimeSeriesDataset

A TimeSeriesDataset is a collection of multiple TimeSeries,
 that behaves similar to a List with additional functionalities.

In [57]:
from timeatlas import TimeSeries, TimeSeriesDataset
import numpy as np

def get_ts():
    return TimeSeries.create(start='2020-10-19', end='2020-10-23', freq="H").fill(value=np.random.randn(97))

tsd = TimeSeriesDataset([get_ts() for i in range(3)])

Statistics will be shown for each TimeSeries
in TimeSeriesDataset with `TimeSeriesDataset.describe()`

In [58]:
tsd.describe()

,minimum,maximum,mean,median,kurtosis,skewness
0,-2.353996,2.178964,0.075252,0.108968,-0.258215,-0.028781
1,-2.177521,3.370040,0.067876,0.081862,1.552037,0.665475
2,-2.904057,2.259880,-0.144704,-0.168570,0.138786,-0.061157


And visualize it with the `TimeSeriesDataset.plot()` function

In [59]:
tsd.plot()